# Planning - Dynamic Programming
https://www.youtube.com/watch?v=dw0sHzE1oAc&t=3278s \\
Naver d2 "Introduction of Deep Reinforcement Learning" 를 참고했습니다.

## Policy Iteration
### 4x4 grid, reward : all -1, action : left, right, up, down

In [1]:
import numpy as np

"""
    0, 0 시작
    index상으로는 0 미만,  3 초과일 수 없음.
"""

state = [0, 0] 


"""
    Function _ get_state 
    : 현재 state에서 action에 의해 변하는 것.
"""
def get_state(state, action): # action , 0: Up, 1 : Down, 2 : Left, 3 : Right
    action_list = [[-1, 0], [1, 0], [0,-1], [0, 1]]
    state[0] += action_list[action][0]
    state[1] += action_list[action][1]
    if state[0] < 0:
        state[0] = 0
    if state[0] > 3 :
        state[0] = 3
    if state[1] < 0:
        state[1] = 0
    if state[1] > 3:
        state[1] = 3

    return state[0], state[1]

"""
    Function _ policy_eval
    : iterative evaluation 부분.

    grid_values는 grid형태 (4 x 4) 의 values table
    
    value table은 그 해당 위치 (state)에서의 미래 상황을 살펴본다. one step or multi step

    action : {0: Up, 1 : Down, 2 : Left, 3 : Right}
    policy : Prob of each states, each action 
    reward : Reward value of each states, each action
"""

def policy_eval(grid_width, grid_height, policy, discount_factor, iters, reward):
    # Function get_state에 정의되어있는 action_list에 index로서 들어간다.
    action = [0, 1, 2 ,3]

    # 다음 Iteration에서의 value를 구하는데 이전 Iteration의 value값이 필요함.
    prev_value_table = np.zeros([grid_height, grid_width]) 

    # 인자로서 받은 iteration만큼 돌려준다. (많이 돌려서 어느 값에 수렴하는 지 확인 -> true value)
    for iter in range(1, iters+1):
        # each iteration에서 각 state의 value를 넣어 줄 수 있는 value_table 구성
        next_value_table = np.zeros([grid_height, grid_width])
        # 모든 state를 모두 한 번씩 조회해보자.
        for i in range(grid_height):
            for j in range(grid_width):
                # State가 시작점 혹은 도착점이면 해당 state는 넘긴다.
                if (i == j) and ((i == 0) or (i == 3)) :
                    continue 

                # 가능한 action들을 모두 고려해서 sum해줘야 해서 action을 통해 for문 돌려준다.
                t_val = 0
                for act in action :
                    i_, j_ = get_state([i, j], act)
                    val = policy[i][j][act]*(reward[i][j][act] + discount_factor * prev_value_table[i_][j_])
            
                    t_val += val

                next_value_table[i][j] = round(t_val,3)
                
        prev_value_table = next_value_table
    return next_value_table

grid_width = 4
grid_height = 4
action = 4  # action : {0: Up, 1 : Down, 2 : Left, 3 : Right}
policy = np.zeros([grid_height, grid_width, action])
reward = np.zeros([grid_height, grid_width, action])
for i in range(grid_height):
    for j in range(grid_width):
        for a in range(action):
            policy[i][j][a] = 0.25
            reward[i][j][a] = -1



for iter in range(10, 101, 10):
    value_table = policy_eval(grid_width, grid_height, policy, discount_factor= 1., iters = iter, reward = reward)
    print("NOW ITERATION : ", iter)
    print(value_table)
    print("\n\n")

print("""
    계속 iteration을 높이다 보면 어느 값에 수렴하는 것을 알 수 있다.
    이를 true value라고 함.
""")

                


    



NOW ITERATION :  10
[[ 0.    -6.138 -8.352 -8.968]
 [-6.138 -7.737 -8.428 -8.352]
 [-8.352 -8.428 -7.737 -6.138]
 [-8.968 -8.352 -6.138  0.   ]]



NOW ITERATION :  20
[[  0.     -9.45  -13.257 -14.454]
 [ -9.45  -12.06  -13.302 -13.257]
 [-13.257 -13.302 -12.06   -9.45 ]
 [-14.454 -13.257  -9.45    0.   ]]



NOW ITERATION :  30
[[  0.    -11.366 -16.096 -17.632]
 [-11.366 -14.562 -16.123 -16.097]
 [-16.096 -16.123 -14.562 -11.366]
 [-17.632 -16.097 -11.366   0.   ]]



NOW ITERATION :  40
[[  0.    -12.475 -17.74  -19.471]
 [-12.475 -16.01  -17.755 -17.74 ]
 [-17.74  -17.755 -16.01  -12.475]
 [-19.471 -17.74  -12.475   0.   ]]



NOW ITERATION :  50
[[  0.    -13.117 -18.691 -20.536]
 [-13.117 -16.847 -18.7   -18.691]
 [-18.691 -18.7   -16.847 -13.117]
 [-20.536 -18.691 -13.117   0.   ]]



NOW ITERATION :  60
[[  0.    -13.489 -19.242 -21.152]
 [-13.489 -17.333 -19.248 -19.242]
 [-19.242 -19.248 -17.333 -13.489]
 [-21.152 -19.242 -13.489   0.   ]]



NOW ITERATION :  70
[[  0.    -1

## Policy Improvement
### Greedy Policy Improvement

In [2]:
"""
    <Action Table>
    0  1  2  3
    4  5  6  7
    8  9  10 11
    12 13 14 15

"""

def greedy_policy(value_table, grid_width, grid_height, action_num):
    action_table = np.zeros([grid_height, grid_width]) # action이 여러개 존재 할 수는 있는데, 하나만 넣자
    action = [0, 1, 2, 3]
    # Greedy 하게 정할 거다. 해당 state를 기준으로 어디로 가는게 제일 효율적인가 를 본다.
    for i in range(grid_height):
        for j in range(grid_width):
            max_value = -100
            for act in action : 
                i_, j_ = get_state([i,j], act)
                value = value_table[i_][j_]
                if max_value < value :
                    arg_ = act
                    max_value = value
            action_table[i,j] = arg_
    return action_table

action_table = greedy_policy(value_table, 4,4,4)



action_dict = {0: "Up", 1 :"Down", 2 : "Left", 3 : "Right"}
action_str_table = []
for i in range(4):
    action_str = []
    for j in range(4):
        if (i == j) and ((i == 0) or (i == 3)):
            action_str.append('T')    
        else :
            action_str.append(action_dict[action_table[i][j]])
    action_str_table.append(action_str)

print("Greedy Policy Improvement")
action_str_table


    

Greedy Policy Improvement


[['T', 'Left', 'Left', 'Down'],
 ['Up', 'Up', 'Down', 'Down'],
 ['Up', 'Up', 'Down', 'Down'],
 ['Up', 'Right', 'Right', 'T']]